# UNDERSTANDING DAN MERGE DATA

In [ ]:
import pandas as pd
train = pd.read_csv("train.csv")

train.head()

import pandas as pd

# Merge genre
genre_map = {
    "Book": "Books & Reference",
    "Books & Reference": "Books & Reference",
    "Magazines & Newspapers": "News & Magazines",
    "News": "News & Magazines",
    "Music": "Music & Audio",
    "Photography": "Photo & Video",
    "Photo & Video": "Photo & Video",
    "Video Players & Editors": "Photo & Video",
    "Navigation": "Maps & Navigation",
    "Social Networking": "Social",
    "Graphics & Design": "Art & Design",
    "Stickers": "Personalization"
}

# Replace genre berdasarkan mapping, tanpa menghapus baris
train["primaryGenreName"] = train["primaryGenreName"].replace(genre_map)

# Lihat hasil unique genre setelah replace
print(train["primaryGenreName"].unique())

# Clean dev country
import numpy as np
# Step 1: Clean downloads
def clean_downloads(val):
    if isinstance(val, str):
        val = val.replace(",", "").replace("+", "")
        if '-' in val:
            low, high = val.split(' - ')
            return (int(low) + int(high)) // 2
        else:
            return int(val)
    return np.nan

train['downloads'] = train['downloads'].apply(clean_downloads)

# Step 2: Replace masking
country_map = {
    'ADDRESS NOT LISTED IN PLAYSTORE': 'UNKNOWN',
    'CANNOT IDENTIFY COUNTRY': 'UNKNOWN',
    'STATUTORY MASKING ENABLED': 'MASKED',
    'PERSONAL DATA, CAN NOT BE PUBLICLY DISCLOSED ACCORDING TO APPLICABLE LAWS.': 'MASKED',
    'TAIWAN, PROVINCE OF CHINA': 'TAIWAN'
}
train['developerCountry'] = train['developerCountry'].replace(country_map)

# Step 3: Hitung top countries
top_download_countries = (
    train.groupby('developerCountry')['downloads']
    .sum()
    .sort_values(ascending=False)
    .head(50)
    .index
)

# Step 4: Mask negara kecil
train['developerCountry'] = train['developerCountry'].apply(
    lambda x: 'OTHER' if (x not in top_download_countries and x not in ['UNKNOWN', 'MASKED']) else x
)

# clean country code drop device
train['countryCode'] = train['countryCode'].fillna(train['deviceType'])
# ubah yang region tidak lazim sebagai country code
region_map = {
    'GLOBAL': 'REGION',
    'EMEA': 'REGION',
    'APAC': 'REGION',
    'LATAM': 'REGION'
}

# Baru apply region_map
train['countryCode'] = train['countryCode'].replace(region_map)

#clean download
import numpy as np
def preprocess_downloads(train, column='downloads'):
    
    
    # 1. Extract min and max values
    def extract_min_max(download_range):
        if pd.isna(download_range):
            return np.nan, np.nan
        
        try:
            parts = download_range.split('-')
            min_val = int(parts[0].strip())
            max_val = int(parts[1].strip())
            
            # Handle reversed cases (e.g., '10 - 1')
            if min_val > max_val:
                return max_val, min_val
            return min_val, max_val
        except:
            return np.nan, np.nan
    
    # Apply extraction function
    train['downloads_min'] = train[column].apply(lambda x: extract_min_max(x)[0])
    train['downloads_max'] = train[column].apply(lambda x: extract_min_max(x)[1])
    
    
    return train

# Helper function to safely parse integers
def try_parse_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return float('inf')  # Return infinity for non-parsable values

# Example usage:
train = preprocess_downloads(train)

# IMPUTASI

In [17]:
import pandas as pd

test = pd.read_csv("test_data_1.csv")
submission = pd.read_csv("submission_format.csv")

# mutlakin kolom test
test["appAge"] = test["appAge"].abs()
test["appAge"].min()


np.float64(0.6333333333333333)

In [18]:
# Pastikan kolom ID di kedua DataFrame bertipe string agar cocok
test["ID"] = test["ID"].astype(str)
submission["ID"] = submission["ID"].astype(str)

# Set index dan atur ulang urutan test berdasarkan urutan ID di submission
test = test.set_index("ID").loc[submission["ID"]].reset_index()

test = test.drop(columns=["Unnamed: 0", "ID"])

# imputasi hasPrivacyLink

In [19]:
# def impute_has_privacy_link(row):
#     if pd.notnull(row['hasPrivacyLink']):
#         return row['hasPrivacyLink']
    
#     # Aturan 1: Aplikasi populer (banyak rating atau banyak download) → diasumsikan True
#     if row['userRatingCount'] > 1000 or row['downloads_max'] > 1e6:
#         return True

#     # Aturan 2: Aplikasi baru (App Age < 60 bulan) → diasumsikan True
#     if row['appAge'] < 60:
#         return True

#     # Aturan 3: Jika memiliki isCorporateEmailScore yang tinggi (profesional) → diasumsikan True
#     if row['isCorporateEmailScore'] == 100:
#         return True

#     # Jika tidak memenuhi semua kriteria → anggap False
#     return False

# # Terapkan ke data
# test['hasPrivacyLink'] = test.apply(impute_has_privacy_link, axis=1)

test["hasPrivacyLink"] = test["countryCode"].apply(lambda x: False if x == "REGION" else True)
test.isna().sum()


developerCountry                      0
countryCode                           0
userRatingCount                       0
primaryGenreName                      0
deviceType                            0
hasPrivacyLink                        0
hasTermsOfServiceLink              1961
hasTermsOfServiceLinkRating        1961
isCorporateEmailScore               440
adSpent                            2426
appAge                               17
averageUserRating                   514
appContentBrandSafetyRating        2671
appDescriptionBrandSafetyRating       0
mfaRating                             0
downloads_min                       881
downloads_max                       881
dtype: int64

# inputasi HasTermofservice

In [20]:
def guess_has_terms(row):
    high_reg_country = ['US', 'GB', 'DE', 'FR', 'CA', 'AU', 'SG']
    email = str(row.get('developerEmail', '')).lower()
    website = str(row.get('developerWebsite', '')).lower()

    # 1. Negara dengan regulasi tinggi
    if row.get('countryCode') in high_reg_country:
        return True

    # 2. Website tersedia dan bukan asal-asalan
    if website and any(domain in website for domain in ['.com', '.org', '.net']):
        return True

    # 3. Email korporat → cenderung profesional
    if email and not email.endswith(('@gmail.com', '@yahoo.com', '@hotmail.com')):
        return True

    # 4. Aplikasi populer → butuh legal protection
    if row.get('downloads_max', 0) >= 100000:
        return True

    return False

test['hasTermsOfServiceLink'] = test.apply(guess_has_terms, axis=1)

test['hasTermsOfServiceLink'].value_counts()


hasTermsOfServiceLink
False    2456
True      544
Name: count, dtype: int64

# inputasi HasTermofservicerating

In [21]:
def rate_terms(val):
    if val == True:
        return 'Low'
    elif val == False:
        return 'High'
    else:
        return 'Missing'

test['hasTermsOfServiceLinkRating'] = test['hasTermsOfServiceLink'].apply(rate_terms)
test.isnull().sum()

test = test.drop(columns="hasTermsOfServiceLink")

# avarage user rating

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
# check anomali avarage --> domain knowledge kalo misal rating count nya 0 otomatis ga ada yang ngerate dan otomatis rata rata nya juga 0
# Filter baris sesuai kondisi
filtered_df = test[(test['userRatingCount'] == 0) & (test['averageUserRating'] != 0)]

# Tampilkan jumlah baris yang memenuhi kondisi
print(f"Jumlah baris dengan userRatingCount = 0 tapi averageUserRating ≠ 0: {len(filtered_df)}")

# Step 1: Jika userRatingCount == 0 dan averageUserRating masih NaN, isi dengan 0
mask = (test['userRatingCount'] == 0) & (test['averageUserRating'].isna())
test.loc[mask, 'averageUserRating'] = 0

# Step 2: Kalau userRatingCount == 0 tapi rating ada isinya (tidak 0), force jadi 0
mask_fix = (test['userRatingCount'] == 0) & (test['averageUserRating'] != 0)
test.loc[mask_fix, 'averageUserRating'] = 0

Jumlah baris dengan userRatingCount = 0 tapi averageUserRating ≠ 0: 616


In [23]:
filtered_df_1 = test[(test['userRatingCount'] != 0) & (test['averageUserRating'] == 0)]

# Tampilkan jumlah baris yang memenuhi kondisi
print(f"Jumlah baris dengan userRatingCount != 0 tapi averageUserRating = 0: {len(filtered_df_1)}")

# kalo misal rating count nya != 0 dan avarage nya 0 force jadi nan
# Step 3: Kalau userRatingCount ≠ 0 tapi averageUserRating = 0 → anggap datanya nggak valid, jadi NaN
mask_force_nan = (test['userRatingCount'] != 0) & (test['averageUserRating'] == 0)
test.loc[mask_force_nan, 'averageUserRating'] = np.nan

Jumlah baris dengan userRatingCount != 0 tapi averageUserRating = 0: 33


# download

In [24]:
# train.isna().sum()
# Pastikan kolom numerik
test['downloads_max'] = pd.to_numeric(test['downloads_max'], errors='coerce')
test['downloads_min'] = pd.to_numeric(test['downloads_min'], errors='coerce')

# Identifikasi baris outlier
outlier_mask = test['downloads_max'] > 100_000_000

# Ubah keduanya
test.loc[outlier_mask, 'downloads_max'] = 50_000_000
test.loc[outlier_mask, 'downloads_min'] = 10_000_000

test["downloads_max"].max()

np.float64(100000000.0)

# encode kolom kategorikal


In [25]:
from sklearn.preprocessing import OrdinalEncoder

# Kolom ordinal yang akan di-encode
ordinal_cols = ["appContentBrandSafetyRating", "appDescriptionBrandSafetyRating", "mfaRating", "hasTermsOfServiceLinkRating"]

# 1. Encode kolom ordinal menggunakan OrdinalEncoder
encoder = OrdinalEncoder()


# Lakukan encoding
test[ordinal_cols] = encoder.fit_transform(test[ordinal_cols])


test.head()

,developerCountry,countryCode,userRatingCount,primaryGenreName,deviceType,hasPrivacyLink,hasTermsOfServiceLinkRating,isCorporateEmailScore,adSpent,appAge,averageUserRating,appContentBrandSafetyRating,appDescriptionBrandSafetyRating,mfaRating,downloads_min,downloads_max
0,UNKNOWN,REGION,2,Medical,GLOBAL,False,0.0,NaN,NaN,81.600000,5.0,NaN,1.0,1.0,NaN,NaN
1,ICELAND,REGION,0,Games,GLOBAL,False,0.0,0.0,0.027742,24.700000,0.0,NaN,1.0,1.0,5000.0,10000.0
2,UNITED STATES,RU,13059,Games,smartphone,True,0.0,99.0,9.249056,40.300000,4.0,NaN,1.0,1.0,NaN,NaN
3,UNKNOWN,REGION,0,Games,GLOBAL,False,0.0,NaN,NaN,39.233333,0.0,NaN,2.0,1.0,NaN,NaN
4,UNKNOWN,REGION,0,Shopping,GLOBAL,False,0.0,NaN,NaN,65.533333,0.0,NaN,1.0,1.0,NaN,NaN


# ubah unknown dan known di dev country dan country code

In [26]:
import numpy as np
import pandas as pd
import numpy as np


# Binary encoding untuk developerCountry (UNKNOWN = 0, lainnya = 1)
test['developerCountry'] = np.where(test['developerCountry'] == 'UNKNOWN', 0, 1)

# Binary encoding untuk countryCode (REGION = 0, lainnya = 1)
test['countryCode'] = np.where(test['countryCode'] == 'REGION', 0, 1)

# Hasil
test.head()

,developerCountry,countryCode,userRatingCount,primaryGenreName,deviceType,hasPrivacyLink,hasTermsOfServiceLinkRating,isCorporateEmailScore,adSpent,appAge,averageUserRating,appContentBrandSafetyRating,appDescriptionBrandSafetyRating,mfaRating,downloads_min,downloads_max
0,0,0,2,Medical,GLOBAL,False,0.0,NaN,NaN,81.600000,5.0,NaN,1.0,1.0,NaN,NaN
1,1,0,0,Games,GLOBAL,False,0.0,0.0,0.027742,24.700000,0.0,NaN,1.0,1.0,5000.0,10000.0
2,1,1,13059,Games,smartphone,True,0.0,99.0,9.249056,40.300000,4.0,NaN,1.0,1.0,NaN,NaN
3,0,0,0,Games,GLOBAL,False,0.0,NaN,NaN,39.233333,0.0,NaN,2.0,1.0,NaN,NaN
4,0,0,0,Shopping,GLOBAL,False,0.0,NaN,NaN,65.533333,0.0,NaN,1.0,1.0,NaN,NaN


# encode genre

In [27]:
genre_encoded = pd.get_dummies(test['primaryGenreName'], prefix='genre')
test = pd.concat([test, genre_encoded], axis=1)
test = test.drop(columns= "primaryGenreName")
test.head()
genre_encoded = pd.get_dummies(test['deviceType'], prefix='genre')
test = pd.concat([test, genre_encoded], axis=1)
test = test.drop(columns= "deviceType")
test.head()

,developerCountry,countryCode,userRatingCount,hasPrivacyLink,hasTermsOfServiceLinkRating,isCorporateEmailScore,adSpent,appAge,averageUserRating,appContentBrandSafetyRating,...,genre_Tools,genre_Travel,genre_Travel & Local,genre_Utilities,genre_Weather,genre_GLOBAL,genre_connected-tv/ott,genre_smartphone,genre_tablet,genre_undetermined
0,0,0,2,False,0.0,NaN,NaN,81.600000,5.0,NaN,...,False,False,False,False,False,True,False,False,False,False
1,1,0,0,False,0.0,0.0,0.027742,24.700000,0.0,NaN,...,False,False,False,False,False,True,False,False,False,False
2,1,1,13059,True,0.0,99.0,9.249056,40.300000,4.0,NaN,...,False,False,False,False,False,False,False,True,False,False
3,0,0,0,False,0.0,NaN,NaN,39.233333,0.0,NaN,...,False,False,False,False,False,True,False,False,False,False
4,0,0,0,False,0.0,NaN,NaN,65.533333,0.0,NaN,...,False,False,False,False,False,True,False,False,False,False


# iterative fill for ad spent, download max download min

In [28]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Kolom yang ingin diimputasi
target_cols = ["adSpent", "downloads_max", "downloads_min"]

# Pilih kolom numerik yang relevan untuk membantu imputasi
# (termasuk target columns dan feature columns yang bisa membantu prediksi)
impute_cols = target_cols + ["userRatingCount", "isCorporateEmailScore", "appAge", "averageUserRating"]

# Pastikan hanya kolom-kolom ini yang dipakai imputasi
impute_df = test[impute_cols].copy()

# Inisialisasi model imputer
iter_imputer = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=10, random_state=42),
    max_iter=10,
    random_state=42
)

# Fit dan transform
imputed_array = iter_imputer.fit_transform(impute_df)

# Masukkan hasil imputasi ke dataframe
imputed_df = pd.DataFrame(imputed_array, columns=impute_cols, index=test.index)

# Update nilai pada kolom imputasi ke data asli
test.update(imputed_df[target_cols])

# lgbm fill 

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import lightgbm as lgb
from tqdm import tqdm

def lgbm_impute_missing(df, ordinal_cols, numerical_cols, n_iterations=3, random_state=42):
    """
    Perform LGBM-based imputation for missing values in ordinal and numerical columns
    
    Parameters:
    - df: DataFrame containing the data
    - ordinal_cols: List of ordinal categorical columns to impute
    - numerical_cols: List of numerical columns to impute
    - n_iterations: Number of iterations for robust imputation
    - random_state: Random seed for reproducibility
    
    Returns:
    - DataFrame with imputed values
    """
    
    # Make a copy of the dataframe to avoid modifying the original
    imputed_df = df.copy()
    
    # Impute ordinal categorical columns
    for col in tqdm(ordinal_cols, desc="Imputing ordinal columns"):
        if col not in imputed_df.columns:
            continue
            
        # Check if column has missing values
        if imputed_df[col].isna().sum() > 0:
            # Split into known and unknown
            known_data = imputed_df[imputed_df[col].notna()]
            unknown_data = imputed_df[imputed_df[col].isna()]
            
            if len(known_data) == 0:
                continue  # Skip if no known values
                
            # Prepare features and target
            X = known_data.drop(columns=ordinal_cols + numerical_cols)
            y = known_data[col]
            
            # Initialize best model and score
            best_model = None
            best_score = 0
            
            # Multiple iterations for robust imputation
            for _ in range(n_iterations):
                # Split into train and validation
                X_train, X_val, y_train, y_val = train_test_split(
                    X, y, test_size=0.2, random_state=random_state+_)
                
                # Train LGBM classifier
                model = lgb.LGBMClassifier(
                    objective='multiclass',
                    random_state=random_state+_
                )
                model.fit(X_train, y_train)
                
                # Evaluate model
                preds = model.predict(X_val)
                score = accuracy_score(y_val, preds)
                
                # Keep the best model
                if score > best_score:
                    best_score = score
                    best_model = model
            
            # Impute missing values
            if best_model is not None:
                imputed_values = best_model.predict(unknown_data.drop(columns=ordinal_cols + numerical_cols))
                imputed_df.loc[imputed_df[col].isna(), col] = imputed_values
    
    # Impute numerical columns
    for col in tqdm(numerical_cols, desc="Imputing numerical columns"):
        if col not in imputed_df.columns:
            continue
            
        # Check if column has missing values
        if imputed_df[col].isna().sum() > 0:
            # Split into known and unknown
            known_data = imputed_df[imputed_df[col].notna()]
            unknown_data = imputed_df[imputed_df[col].isna()]
            
            if len(known_data) == 0:
                continue  # Skip if no known values
                
            # Prepare features and target
            X = known_data.drop(columns=ordinal_cols + numerical_cols)
            y = known_data[col]
            
            # Initialize best model and score
            best_model = None
            best_score = float('inf')
            
            # Multiple iterations for robust imputation
            for _ in range(n_iterations):
                # Split into train and validation
                X_train, X_val, y_train, y_val = train_test_split(
                    X, y, test_size=0.2, random_state=random_state+_)
                
                # Train LGBM regressor
                model = lgb.LGBMRegressor(
                    objective='regression',
                    random_state=random_state+_
                )
                model.fit(X_train, y_train)
                
                # Evaluate model
                preds = model.predict(X_val)
                score = np.sqrt(mean_squared_error(y_val, preds))
                
                # Keep the best model
                if score < best_score:
                    best_score = score
                    best_model = model
            
            # Impute missing values
            if best_model is not None:
                imputed_values = best_model.predict(unknown_data.drop(columns=ordinal_cols + numerical_cols))
                imputed_df.loc[imputed_df[col].isna(), col] = imputed_values
    
    return imputed_df

# Contoh penggunaan:
ordinal_cols = ["appContentBrandSafetyRating","appDescriptionBrandSafetyRating", "mfaRating"]
numerical_cols = ["userRatingCount", "adSpent", "appAge", "averageUserRating", "downloads_min", "downloads_max", "isCorporateEmailScore"]
test = lgbm_impute_missing(test, ordinal_cols, numerical_cols)

Imputing ordinal columns: 100%|██████████| 3/3 [00:00<00:00, 33.71it/s]


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000013 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 263, number of used features: 8
[LightGBM] [Info] Start training from score -4.879007
[LightGBM] [Info] Start training from score -1.018277
[LightGBM] [Info] Start training from score -0.460166
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

Imputing numerical columns:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000121 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 62
[LightGBM] [Info] Number of data points in the train set: 2386, number of used features: 31
[LightGBM] [Info] Start training from score 55.968217
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000121 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 64
[LightGBM] [Info] Number of data points in the train set: 2386, number of used features: 32
[LightGBM] [Info] Start training from score 55.810031
[LightGBM] [Warning] Found white

Imputing numerical columns:  43%|████▎     | 3/7 [00:00<00:00, 25.42it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000105 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 58
[LightGBM] [Info] Number of data points in the train set: 2373, number of used features: 29
[LightGBM] [Info] Start training from score 1.941846
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 62
[LightGBM] [Info] Number of data points in the train set: 2373, number of used features: 31
[LightGBM] [Info] Start training from score 1.941424


Imputing numerical columns:  86%|████████▌ | 6/7 [00:00<00:00, 23.88it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000105 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 60
[LightGBM] [Info] Number of data points in the train set: 2373, number of used features: 30
[LightGBM] [Info] Start training from score 1.911504
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000098 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 58
[LightGBM] [Info] Number of data points in the train set: 2048, number of used features: 29
[LightGBM] [Info] Start training from score 46.116211
[LightGBM] [Warning] Found whites

Imputing numerical columns: 100%|██████████| 7/7 [00:00<00:00, 18.94it/s]


In [ ]:
test.isnull().sum()
test.to_csv("test_filloke6.csv", index=False)
test[["userRatingCount", "isCorporateEmailScore", "appAge", "averageUserRating", "downloads_min", "downloads_max", "adSpent"]].min()


userRatingCount          0.000000
isCorporateEmailScore    0.000000
appAge                   0.633333
averageUserRating        0.000000
downloads_min            0.000000
downloads_max            1.000000
adSpent                  0.007450
dtype: float64

# Modelling

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

## 1. Persiapan Data
def prepare_data(df):
    """Preprocessing data untuk modeling"""
    # Handle kolom Downloads
    if 'Downloads' in df.columns:
        downloads_split = df['Downloads'].str.extract(r'(\d+)\s*-\s*(\d+)')
        df['downloads_min'] = pd.to_numeric(downloads_split[0], errors='coerce')
        df['downloads_max'] = pd.to_numeric(downloads_split[1], errors='coerce')
        df.drop('Downloads', axis=1, inplace=True)

    # Convert boolean columns
    for col in ['hasPrivacyLink', 'hasTermsOfServiceLink']:
        if col in df.columns:
            df[col] = df[col].astype(int)

    # Encode kolom kategorikal
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    for col in cat_cols:
        if col != 'coppaRisk':
            df[col] = LabelEncoder().fit_transform(df[col].astype(str).fillna('MISSING'))

    return df


## 2. Split Data
X = train.drop(columns=['coppaRisk'])
y = train['coppaRisk']

# Stratified split untuk menjaga distribusi imbalance
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

## 3. Modeling dengan XGBoost
# Hitung rasio imbalance
scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)

# Buat pipeline dengan SMOTE dan XGBoost
model = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('xgb', XGBClassifier(
        objective='binary:logistic',
        eval_metric='auc',
        learning_rate=0.05,
        max_depth=5,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        n_jobs=-1
    ))
])

# Training model
model.fit(X_train, y_train)

## 4. Evaluasi Model
# Prediksi probabilities
y_pred_proba = model.predict_proba(X_val)[:, 1]

# Hitung AUC Score
auc_score = roc_auc_score(y_val, y_pred_proba)
print(f"\nValidation AUC Score: {auc_score:.4f}")

# Classification report dengan threshold 0.5
y_pred = (y_pred_proba >= 0.5).astype(int)
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# Confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## 5. Cross Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for train_idx, val_idx in cv.split(X, y):
    X_train_cv, X_val_cv = X.iloc[train_idx], X.iloc[val_idx]
    y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]

    model_cv = Pipeline([
        ('smote', SMOTE(random_state=42)),
        ('xgb', XGBClassifier(
            objective='binary:logistic',
            eval_metric='auc',
            learning_rate=0.05,
            max_depth=5,
            min_child_weight=3,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.1,
            reg_lambda=0.1,
            scale_pos_weight=scale_pos_weight,
            random_state=42,
            n_jobs=-1
        ))
    ])

    model_cv.fit(X_train_cv, y_train_cv)
    y_pred_proba_cv = model_cv.predict_proba(X_val_cv)[:, 1]
    cv_scores.append(roc_auc_score(y_val_cv, y_pred_proba_cv))

print(f"\nCross-Validation AUC Scores: {cv_scores}")
print(f"Mean CV AUC: {np.mean(cv_scores):.4f} (±{np.std(cv_scores):.4f})")

## 6. Feature Importance
# Dapatkan model XGBoost dari pipeline
xgb_model = model.named_steps['xgb']

# Plot importance
plt.figure(figsize=(10, 6))
sorted_idx = xgb_model.feature_importances_.argsort()
plt.barh(X.columns[sorted_idx][-20:], xgb_model.feature_importances_[sorted_idx][-20:])
plt.xlabel("XGBoost Feature Importance")
plt.show()


# Prediksi probabilitas dengan XGBRegressor
test_pred_proba = model.predict(test)

# Thresholding (misalnya threshold 0.5, bisa disesuaikan)
test_pred = (test_pred_proba >= 0.5).astype(int)

# Simpan ke DataFrame hasil akhir
output = pd.DataFrame({
    'id': test['id'] if 'id' in test.columns else np.arange(len(test)),
    'coppaRisk_predicted': test_pred,
    'coppaRisk_proba': test_pred_proba
})

# Tampilkan hasil
print(output.head())

# Export ke file CSV
output.to_csv("submission.csv", index=False)
print("\n✅ File 'submission.csv' berhasil disimpan.")




# Ambil kolom ID dari submission_format
id_column = submission_format['ID']

# Gabungkan jadi DataFrame akhir
final_output = pd.DataFrame({
    'ID': id_column,
    'coppaRisk': test_pred_proba
})

# Cek hasil
print(final_output.head())
final_output.to_csv("submission.csv", index = False)